# RAG 성능 평가 지표 완전 가이드

## 개요

이 노트북에서는 RAG(Retrieval-Augmented Generation) 시스템의 검색 성능을 측정하는 주요 지표들을 **이론과 실습**을 통해 완벽하게 마스터합니다.

| 주제 | 내용 |
|------|------|
| 기본 지표 | MRR, nDCG, Precision@K, Recall@K |
| 고급 지표 | F1-Score@K, MAP, Hit Rate |
| 구현 | Python으로 직접 구현 |
| 시각화 | 지표별 시각화 및 비교 |
| 실전 | 실제 RAG 시스템 평가 |

## 학습 목표

1. 각 평가 지표의 개념과 수식 이해하기
2. Python으로 평가 지표 직접 구현하기
3. 시각화를 통한 직관적 이해
4. 실제 RAG 시스템 평가에 적용하기
5. 평가 데이터셋 구축 방법 익히기

---

## 0. 필요한 라이브러리 설치 및 임포트

In [10]:
# 필요한 패키지 설치
!pip install numpy pandas matplotlib plotly scikit-learn


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'AppleGothic' if plt.rcParams['font.family'] == 'sans-serif' else plt.rcParams['font.family']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 모든 라이브러리가 로드되었습니다.")

✅ 모든 라이브러리가 로드되었습니다.


---

## 1. MRR (Mean Reciprocal Rank)

### 핵심 아이디어
"정답이 몇 번째에 나왔나?"를 점수로 바꾼 것입니다.

### 점수 계산 방식

| 정답 위치 | 점수 |
|-----------|------|
| 1등 | 1/1 = **1.0** |
| 2등 | 1/2 = **0.5** |
| 3등 | 1/3 = **0.33** |
| 10등 | 1/10 = **0.1** |

정답이 위에 있을수록 점수가 높습니다.

### 공식

```
MRR = (1/Q) × Σ(1/rank_i)
```

- Q: 전체 쿼리 수
- rank_i: i번째 쿼리에서 첫 번째 정답의 순위

### 예시

RAG 시스템에 3번 질문했을 때:

| 질문 | 정답 위치 | 점수 |
|------|-----------|------|
| "김치 만드는 법" | 1등 | 1.0 |
| "서울 날씨" | 3등 | 0.33 |
| "파이썬 설치" | 2등 | 0.5 |

**MRR** = (1.0 + 0.33 + 0.5) / 3 = **0.61**

### 해석

- **1.0**: 완벽함 (항상 정답이 1등)
- **0.5 이상**: 괜찮은 성능
- **0.5 미만**: 개선 필요


In [12]:
def calculate_mrr(rankings: List[int]) -> float:
    """
    MRR (Mean Reciprocal Rank) 계산
    
    Args:
        rankings: 각 쿼리에서 첫 번째 정답의 순위 리스트
                 (정답이 없으면 0 또는 큰 수)
    
    Returns:
        MRR 점수
    """
    reciprocal_ranks = []
    
    for rank in rankings:
        if rank > 0:  # 정답이 있는 경우
            reciprocal_ranks.append(1.0 / rank)
        else:  # 정답이 없는 경우
            reciprocal_ranks.append(0.0)
    
    return np.mean(reciprocal_ranks)

# 예제: RAG 시스템에 3번 질문했을 때
example_rankings = [
    1,  # "김치 만드는 법" - 정답이 1등
    3,  # "서울 날씨" - 정답이 3등
    2,  # "파이썬 설치" - 정답이 2등
]

mrr_score = calculate_mrr(example_rankings)
print(f"예제 쿼리들의 순위: {example_rankings}")
print(f"각 쿼리의 점수: {[1/r for r in example_rankings]}")
print(f"MRR = {mrr_score:.3f}")
print(f"\n해석: {mrr_score:.3f}는 {'좋은' if mrr_score > 0.5 else '개선이 필요한'} 성능입니다.")

예제 쿼리들의 순위: [1, 3, 2]
각 쿼리의 점수: [1.0, 0.3333333333333333, 0.5]
MRR = 0.611

해석: 0.611는 좋은 성능입니다.


## 2. nDCG (Normalized Discounted Cumulative Gain)

### 핵심 아이디어
"관련성 높은 결과가 위에 있을수록 좋다"를 점수로 바꾼 것입니다."

"관련성이 높은 결과가 상단에 위치할수록 가산점을 주되, 순위가 뒤로 밀릴수록 그 가치를 로그 함수로 나누어 감점(Discount)하는 방식"
### MRR과의 차이점

| | MRR | nDCG |
|---|-----|------|
| 정답 개수 | 가장 먼저 나온 **1개** 만 반영 | 검색된 **여러 개** 의 답을 모두 반영 |
| 관련성 | 정답/오답 (0 or 1) | **등급 평가** (0, 1, 2, 3...) |
| 주요 목적 | 첫번째 정답이 얼마나 빨리 나오는가? | 전체적인 검색 결과의 질과 순서가 적절한가? |

### 계산 과정 (3단계)

#### 1단계: 관련성 점수 매기기

검색 결과 5개가 있다고 가정:

| 순위 | 문서 | 관련성 |
|------|------|--------|
| 1등 | 문서A | 3 (매우 관련) |
| 2등 | 문서B | 1 (조금 관련) |
| 3등 | 문서C | 2 (관련) |
| 4등 | 문서D | 0 (무관) |
| 5등 | 문서E | 1 (조금 관련) |

#### 2단계: DCG 계산

아래로 갈수록 점수를 **깎습니다** (할인 적용):

```
DCG = 3/log₂(2) + 1/log₂(3) + 2/log₂(4) + 0/log₂(5) + 1/log₂(6)
    = 3 + 0.63 + 1 + 0 + 0.39 = 5.02
```

#### 3단계: 정규화 (nDCG)

검색 시스템마다 뽑아주는 결과의 개수가 다르므로, 이를 비교 가능하도록 0~1 사이의 값으로 변환합니다. 

이를 위해 이상적인 배치(IDCG) 값으로 나누어줍니다.IDCG (Ideal DCG): 현재 결과들을 '가장 관련성이 높은 순'으로 

재배치했을 때의 DCG 값. (위 예시에서는 3, 2, 1, 1, 0 순서)

계산: $nDCG = \frac{DCG}{IDCG}$

**이상적인 순서** (3, 2, 1, 1, 0)로 정렬했을 때의 DCG(IDCG)를 구하고 나눕니다:

```
nDCG = DCG / IDCG = 5.02 / 6.39 = 0.79
```

### 관련성 점수는 어떻게 부여할까?

위 예시에서 관련성 점수(3, 1, 2, 0, 1)는 어떻게 결정할까요? 세 가지 대표적인 방법이 있습니다.

#### 방법 1: Binary Relevance (이진 관련성)

가장 단순한 방법으로, 관련 여부만 판단합니다.

| 점수 | 의미 |
|------|------|
| 1 | 관련 있음 |
| 0 | 관련 없음 |

**장점**: 레이블링이 쉽고 빠름
**단점**: "매우 관련" vs "조금 관련"을 구분하지 못함
**사용 시점**: 정답이 명확한 경우 (예: FAQ, 정의 검색)

#### 방법 2: Graded Relevance (등급별 관련성)

전문가가 여러 등급으로 관련성을 평가합니다.

| 점수 | 의미 | 예시 |
|------|------|------|
| 3 | 완벽히 답변 | 질문에 대한 정확한 답이 포함됨 |
| 2 | 매우 관련 | 답변에 직접적으로 도움됨 |
| 1 | 부분 관련 | 일부 관련 정보 포함 |
| 0 | 무관 | 질문과 상관없음 |

**장점**: 세밀한 품질 평가 가능
**단점**: 레이블링 비용 높음, 평가자 간 불일치 가능
**사용 시점**: 고품질 평가 데이터셋 구축 시

#### 방법 3: LLM 기반 자동 평가

GPT-4 등 LLM을 사용하여 관련성 점수를 자동으로 부여합니다.

**프롬프트 예시**:
```
질문: {query}
문서: {document}

이 문서가 질문에 얼마나 관련이 있는지 0~3점으로 평가하세요:
- 3: 완벽히 답변 (정확한 답 포함)
- 2: 매우 관련 (답변에 도움됨)
- 1: 부분 관련 (일부 정보만 관련)
- 0: 무관

점수만 숫자로 출력하세요.
```

**장점**: 대량 평가 가능, 일관성 있는 평가
**단점**: API 비용 발생, LLM 편향 가능성
**사용 시점**: 대규모 데이터셋 자동 평가 시

#### 실전 권장사항

| 상황 | 권장 방법 |
|------|-----------|
| 소규모 (100개 미만) | Graded Relevance (전문가 평가) |
| 중규모 (100~1000개) | Binary + 샘플 Graded 검증 |
| 대규모 (1000개 이상) | LLM 기반 + 샘플링 검증 |

### 해석

- **1.0**: 완벽한 순서 (가장 관련 높은 것이 맨 위)
- **0.8 이상**: 좋은 성능
- **0.5 미만**: 순서 개선 필요

### 사용 시점

- 강점: 검색 결과의 '양'과 '질'을 모두 평가할 수 있어 현대 검색 엔진 및 추천 시스템에서 가장 표준적으로 사용되는 지표입니다.
- RAG에서의 활용: 사용자의 질문에 대해 리트리버(Retriever)가 가져온 상위 K개의 문서들이 얼마나 질문과 밀접하고 순서가 적절한지 평가할 때 필수적입니다.

---

In [3]:
def calculate_dcg(relevances: List[float], k: int = None) -> float:
    """
    DCG (Discounted Cumulative Gain) 계산
    
    Args:
        relevances: 각 문서의 관련성 점수 리스트
        k: 상위 k개만 고려 (None이면 전체)
    
    Returns:
        DCG 점수
    """
    if k is None:
        k = len(relevances)
    
    dcg = 0.0
    for i in range(min(k, len(relevances))):
        # 위치 i+1의 할인율 적용 (i는 0부터 시작)
        dcg += relevances[i] / np.log2(i + 2)  # i+2는 position이 1부터 시작하기 때문
    
    return dcg

def calculate_ndcg(relevances: List[float], k: int = None) -> float:
    """
    nDCG (Normalized DCG) 계산
    
    Args:
        relevances: 각 문서의 관련성 점수 리스트
        k: 상위 k개만 고려
    
    Returns:
        nDCG 점수 (0~1)
    """
    # 실제 DCG 계산
    dcg = calculate_dcg(relevances, k)
    
    # 이상적인 순서로 정렬하여 IDCG 계산
    ideal_relevances = sorted(relevances, reverse=True)
    idcg = calculate_dcg(ideal_relevances, k)
    
    # IDCG가 0이면 nDCG도 0
    if idcg == 0:
        return 0.0
    
    return dcg / idcg

# 예제: 검색 결과 5개의 관련성 점수
relevances = [3, 1, 2, 0, 1]  # 실제 순서
ideal_order = sorted(relevances, reverse=True)  # [3, 2, 1, 1, 0]

dcg = calculate_dcg(relevances)
idcg = calculate_dcg(ideal_order)
ndcg = calculate_ndcg(relevances)

print("📊 nDCG 계산 과정:")
print(f"\n1. 실제 관련성 순서: {relevances}")
print(f"2. 이상적인 순서: {ideal_order}")
print(f"\n3. DCG = {dcg:.3f}")
print(f"4. IDCG = {idcg:.3f}")
print(f"5. nDCG = DCG/IDCG = {ndcg:.3f}")
print(f"\n해석: {ndcg:.3f}는 {'좋은' if ndcg > 0.8 else '개선이 필요한'} 순서입니다.")

📊 nDCG 계산 과정:

1. 실제 관련성 순서: [3, 1, 2, 0, 1]
2. 이상적인 순서: [3, 2, 1, 1, 0]

3. DCG = 5.018
4. IDCG = 5.193
5. nDCG = DCG/IDCG = 0.966

해석: 0.966는 좋은 순서입니다.


### nDCG 시각화

In [4]:
# 여러 검색 결과의 nDCG 비교
scenarios = {
    '완벽한 순서': [3, 2, 1, 1, 0],
    '좋은 순서': [3, 1, 2, 0, 1],
    '나쁜 순서': [0, 1, 1, 2, 3],
    '랜덤 순서': [1, 0, 3, 1, 2]
}

results = []
for name, rels in scenarios.items():
    ndcg = calculate_ndcg(rels)
    results.append({'시나리오': name, 'nDCG': ndcg, '관련성 순서': str(rels)})

df_ndcg = pd.DataFrame(results)

# 막대 그래프
fig = px.bar(df_ndcg, x='시나리오', y='nDCG', 
             hover_data=['관련성 순서'],
             title='시나리오별 nDCG 점수 비교',
             color='nDCG',
             color_continuous_scale='RdYlGn')

fig.update_layout(yaxis_range=[0, 1.1])
fig.add_hline(y=0.8, line_dash="dash", line_color="green", 
              annotation_text="좋은 성능 (0.8)")
fig.add_hline(y=0.5, line_dash="dash", line_color="orange", 
              annotation_text="개선 필요 (0.5)")

fig.show()

print("\n📊 결과 테이블:")
print(df_ndcg.to_string(index=False))


📊 결과 테이블:
  시나리오     nDCG          관련성 순서
완벽한 순서 1.000000 [3, 2, 1, 1, 0]
 좋은 순서 0.966345 [3, 1, 2, 0, 1]
 나쁜 순서 0.607187 [0, 1, 1, 2, 3]
 랜덤 순서 0.713405 [1, 0, 3, 1, 2]


---
## 3. Precision@K (정밀도)

### 핵심 아이디어
"가져온 K개 중에 정답이 몇 개?"

### 공식

```
Precision@K = 상위 K개 중 정답 수 / K
```

### 예시 (K=5)

검색 결과 상위 5개: ✓ ✗ ✓ ✓ ✗ (정답 3개)

```
Precision@5 = 3/5 = 0.6 (60%)
```

### 관점
**품질** 중심 - 쓸모없는 결과 줄이기

---

## 4. Recall@K (재현율)

### 핵심 아이디어
"전체 정답 중에 K개 안에 몇 개 들어왔나?"

### 공식

```
Recall@K = 상위 K개 중 정답 수 / 전체 정답 수
```

### 예시 (K=5, 전체 정답 10개)

검색 결과 상위 5개 중 정답 3개

```
Recall@5 = 3/10 = 0.3 (30%)
```

### 관점
**커버리지** 중심 - 정답 놓치지 않기

---

## 5. Precision@K vs Recall@K 비교

### 개념 비교

| | Precision@K | Recall@K |
|---|-------------|----------|
| 질문 | K개 중 정답 비율? | 전체 정답 중 찾은 비율? |
| 분모 | K (고정) | 전체 정답 수 |
| 관점 | **품질** (쓸모없는 것 줄이기) | **커버리지** (놓치지 않기) |

### 실제 예시

전체 관련 문서가 **8개**인 질문에 대해 상위 5개를 가져왔을 때:

| 순위 | 결과 |
|------|------|
| 1 | ✓ 정답 |
| 2 | ✓ 정답 |
| 3 | ✗ 오답 |
| 4 | ✓ 정답 |
| 5 | ✗ 오답 |

- **Precision@5** = 3/5 = 0.6 → "가져온 것 중 60%가 쓸모있음"
- **Recall@5** = 3/8 = 0.375 → "전체 정답의 37.5%를 찾음"

### 트레이드오프

K를 늘리면 Recall은 올라가지만 Precision은 내려가는 경향이 있습니다.

---



In [5]:
def calculate_precision_at_k(retrieved: List[int], relevant: List[int], k: int) -> float:
    """
    Precision@K 계산
    
    Args:
        retrieved: 검색된 문서 ID 리스트 (순서대로)
        relevant: 관련 문서 ID 리스트
        k: 상위 k개 고려
    
    Returns:
        Precision@K 점수
    """
    retrieved_k = retrieved[:k]
    relevant_set = set(relevant)
    
    # 상위 k개 중 관련 문서 개수
    relevant_in_k = sum(1 for doc in retrieved_k if doc in relevant_set)
    
    return relevant_in_k / k if k > 0 else 0

def calculate_recall_at_k(retrieved: List[int], relevant: List[int], k: int) -> float:
    """
    Recall@K 계산
    
    Args:
        retrieved: 검색된 문서 ID 리스트 (순서대로)
        relevant: 관련 문서 ID 리스트
        k: 상위 k개 고려
    
    Returns:
        Recall@K 점수
    """
    retrieved_k = retrieved[:k]
    relevant_set = set(relevant)
    
    # 상위 k개 중 관련 문서 개수
    relevant_in_k = sum(1 for doc in retrieved_k if doc in relevant_set)
    
    return relevant_in_k / len(relevant_set) if len(relevant_set) > 0 else 0

# 예제: RAG 검색 결과
retrieved_docs = [1, 3, 5, 7, 9, 2, 4, 6, 8, 10]  # 검색된 문서 ID (순서대로)
relevant_docs = [1, 2, 5, 8, 11, 13, 15, 17]  # 실제 관련 문서 ID

# K=5일 때 계산
k = 5
precision_5 = calculate_precision_at_k(retrieved_docs, relevant_docs, k)
recall_5 = calculate_recall_at_k(retrieved_docs, relevant_docs, k)

print(f"📊 검색 결과 분석 (K={k}):")
print(f"\n검색된 상위 {k}개: {retrieved_docs[:k]}")
print(f"전체 관련 문서: {relevant_docs}")
print(f"상위 {k}개 중 정답: {[doc for doc in retrieved_docs[:k] if doc in relevant_docs]}")
print(f"\nPrecision@{k} = {precision_5:.2f} ({precision_5*100:.0f}%)")
print(f"  → 가져온 {k}개 중 {precision_5*k:.0f}개가 정답")
print(f"\nRecall@{k} = {recall_5:.2f} ({recall_5*100:.0f}%)")
print(f"  → 전체 정답 {len(relevant_docs)}개 중 {recall_5*len(relevant_docs):.0f}개를 찾음")

📊 검색 결과 분석 (K=5):

검색된 상위 5개: [1, 3, 5, 7, 9]
전체 관련 문서: [1, 2, 5, 8, 11, 13, 15, 17]
상위 5개 중 정답: [1, 5]

Precision@5 = 0.40 (40%)
  → 가져온 5개 중 2개가 정답

Recall@5 = 0.25 (25%)
  → 전체 정답 8개 중 2개를 찾음


### Precision-Recall 트레이드오프 시각화

In [7]:
# K값 변화에 따른 Precision과 Recall 변화
k_values = range(1, 11)
precisions = []
recalls = []

for k in k_values:
    p = calculate_precision_at_k(retrieved_docs, relevant_docs, k)
    r = calculate_recall_at_k(retrieved_docs, relevant_docs, k)
    precisions.append(p)
    recalls.append(r)

# 그래프 생성
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(k_values), y=precisions,
    mode='lines+markers',
    name='Precision@K',
    marker=dict(size=8)
))

fig.add_trace(go.Scatter(
    x=list(k_values), y=recalls,
    mode='lines+markers',
    name='Recall@K',
    marker=dict(size=8)
))

fig.update_layout(
    title='Precision@K vs Recall@K: 트레이드오프',
    xaxis_title='K (검색 결과 개수)',
    yaxis_title='점수',
    hovermode='x unified',
    yaxis_range=[0, 1]
)

fig.show()

print("💡 인사이트:")
print("- K가 증가하면 Recall은 올라가지만 Precision은 떨어집니다.")
print("- 적절한 K값 선택이 중요합니다!")

💡 인사이트:
- K가 증가하면 Recall은 올라가지만 Precision은 떨어집니다.
- 적절한 K값 선택이 중요합니다!


---

## 6. F1-Score@K

### 📚 개념

Precision과 Recall의 조화평균으로, 두 지표의 균형을 측정합니다.

### 🔢 공식

$$F1@K = 2 \times \frac{Precision@K \times Recall@K}{Precision@K + Recall@K}$$

In [ ]:
def calculate_f1_at_k(retrieved: List[int], relevant: List[int], k: int) -> float:
    """
    F1-Score@K 계산
    
    Args:
        retrieved: 검색된 문서 ID 리스트
        relevant: 관련 문서 ID 리스트
        k: 상위 k개 고려
    
    Returns:
        F1@K 점수
    """
    precision = calculate_precision_at_k(retrieved, relevant, k)
    recall = calculate_recall_at_k(retrieved, relevant, k)
    
    if precision + recall == 0:
        return 0
    
    return 2 * (precision * recall) / (precision + recall)

# K값별 F1-Score 계산
f1_scores = []
for k in k_values:
    f1 = calculate_f1_at_k(retrieved_docs, relevant_docs, k)
    f1_scores.append(f1)

# 모든 지표 함께 시각화
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(k_values), y=precisions, mode='lines+markers', name='Precision@K'))
fig.add_trace(go.Scatter(x=list(k_values), y=recalls, mode='lines+markers', name='Recall@K'))
fig.add_trace(go.Scatter(x=list(k_values), y=f1_scores, mode='lines+markers', name='F1-Score@K',
                        line=dict(width=3)))

fig.update_layout(
    title='검색 성능 지표 종합 비교',
    xaxis_title='K (검색 결과 개수)',
    yaxis_title='점수',
    hovermode='x unified',
    yaxis_range=[0, 1]
)

fig.show()

# 최적 K 찾기
best_k = k_values[np.argmax(f1_scores)]
best_f1 = max(f1_scores)

print(f"\n🎯 최적 K값: {best_k} (F1-Score: {best_f1:.3f})")
print(f"   - Precision@{best_k}: {precisions[best_k-1]:.3f}")
print(f"   - Recall@{best_k}: {recalls[best_k-1]:.3f}")

---

## 7. Keyword Coverage (키워드 커버리지)

### 핵심 아이디어
"검색된 문서에서 핵심 키워드가 얼마나 발견되는가?"

LLM 기반 평가 없이 빠르게 검색 품질을 확인할 수 있는 휴리스틱 지표입니다.

### 공식

$$Keyword\ Coverage = \frac{\text{발견된 키워드 수}}{\text{전체 키워드 수}}$$

### 특징

| 장점 | 단점 |
|------|------|
| 빠른 계산 (LLM 불필요) | 의미적 유사성 미반영 |
| 비용 없음 | 동의어 처리 어려움 |
| 직관적 해석 | 키워드 선정이 주관적 |

### 사용 시점
- 빠른 검색 품질 확인이 필요할 때
- LLM API 호출 비용을 아끼고 싶을 때
- 기본적인 관련성 체크가 필요할 때
- 디버깅 및 개발 단계에서

In [13]:
def calculate_keyword_coverage(retrieved_content: str, keywords: List[str]) -> float:
    """
    키워드 커버리지 계산

    Args:
        retrieved_content: 검색된 문서들의 내용 (연결된 텍스트)
        keywords: 찾아야 할 키워드 리스트

    Returns:
        커버리지 비율 (0~1)
    """
    if not keywords:
        return 1.0  # 키워드가 없으면 100%로 간주

    content_lower = retrieved_content.lower()
    found_count = sum(1 for kw in keywords if kw.lower() in content_lower)

    return found_count / len(keywords)


# 예시
document_content = """
Python에서 리스트를 정렬하는 방법은 두 가지가 있습니다.
sort() 메서드는 원본 리스트를 직접 수정합니다.
sorted() 함수는 새로운 정렬된 리스트를 반환합니다.
"""

keywords = ["sort", "sorted", "리스트", "정렬", "reverse"]  # reverse는 없음

coverage = calculate_keyword_coverage(document_content, keywords)
print(f"키워드: {keywords}")
print(f"Keyword Coverage: {coverage:.1%}")
print(f"\n해석: 5개 중 4개 키워드가 발견됨 (reverse 누락)")

키워드: ['sort', 'sorted', '리스트', '정렬', 'reverse']
Keyword Coverage: 80.0%

해석: 5개 중 4개 키워드가 발견됨 (reverse 누락)


## 8. 지표 선택 가이드 (간략)

| 지표 | 사용 상황 |
|------|-----------|
| **MRR** | 첫 번째 정답 위치가 중요할 때 (단일 정답) |
| **nDCG** | 여러 결과의 전체 품질과 순서가 중요할 때 |
| **Precision@K** | 검색 결과에 노이즈가 적어야 할 때 |
| **Recall@K** | 정답을 놓치면 안 될 때 |
| **Keyword Coverage** | 빠른 검색 품질 확인이 필요할 때 |

---

## 9. 요약

| 지표 | 핵심 질문 | 범위 |
|------|-----------|------|
| MRR | 첫 정답이 몇 번째? | 0 ~ 1 |
| nDCG | 관련성 높은 게 위에 있나? | 0 ~ 1 |
| Precision@K | K개 중 정답 비율? | 0 ~ 1 |
| Recall@K | 전체 정답 중 찾은 비율? | 0 ~ 1 |
| Keyword Coverage | 키워드가 얼마나 포함? | 0 ~ 1 |

모든 지표는 **1에 가까울수록 좋은 성능**을 의미합니다.

---

## 10. 평가 데이터셋 구축 방법

실제 RAG 시스템을 평가하기 위한 데이터셋 구축 방법을 알아봅시다.

In [6]:
class RAGTestDataset:
    """
    RAG 평가용 데이터셋 클래스
    """
    
    def __init__(self):
        self.queries = []
        self.documents = {}
        self.relevance_judgments = {}
    
    def add_document(self, doc_id: int, content: str, metadata: dict = None):
        """
        문서 추가
        """
        self.documents[doc_id] = {
            'content': content,
            'metadata': metadata or {}
        }
    
    def add_query(self, query_id: int, query_text: str, relevant_docs: List[int], 
                  relevance_scores: Dict[int, float] = None):
        """
        쿼리와 관련 문서 추가
        
        Args:
            query_id: 쿼리 ID
            query_text: 쿼리 텍스트
            relevant_docs: 관련 문서 ID 리스트
            relevance_scores: 문서별 관련성 점수 (선택적)
        """
        self.queries.append({
            'id': query_id,
            'text': query_text
        })
        
        self.relevance_judgments[query_id] = {
            'relevant_docs': relevant_docs,
            'scores': relevance_scores or {doc_id: 1.0 for doc_id in relevant_docs}
        }
    
    def get_statistics(self) -> dict:
        """
        데이터셋 통계
        """
        total_relevant = sum(len(j['relevant_docs']) 
                           for j in self.relevance_judgments.values())
        
        return {
            'num_queries': len(self.queries),
            'num_documents': len(self.documents),
            'avg_relevant_per_query': total_relevant / len(self.queries) if self.queries else 0,
            'total_relevance_judgments': total_relevant
        }
    
    def export_to_json(self, filepath: str):
        """
        JSON으로 내보내기
        """
        import json
        
        data = {
            'queries': self.queries,
            'documents': self.documents,
            'relevance_judgments': self.relevance_judgments
        }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

# 평가 데이터셋 구축 예제
dataset = RAGTestDataset()

# 문서 추가
documents = [
    (1, "Python은 간단하고 읽기 쉬운 프로그래밍 언어입니다."),
    (2, "리스트는 Python의 기본 자료구조 중 하나입니다."),
    (3, "정렬 알고리즘에는 퀵소트, 머지소트, 힙소트 등이 있습니다."),
    (4, "머신러닝은 데이터로부터 패턴을 학습하는 기술입니다."),
    (5, "딥러닝은 신경망을 여러 층으로 쌓은 머신러닝 기법입니다."),
]

for doc_id, content in documents:
    dataset.add_document(doc_id, content)

# 쿼리와 관련성 판단 추가
dataset.add_query(
    query_id=1,
    query_text="Python 리스트 정렬 방법",
    relevant_docs=[2, 3],
    relevance_scores={2: 1.0, 3: 0.8}
)

dataset.add_query(
    query_id=2,
    query_text="머신러닝과 딥러닝의 차이",
    relevant_docs=[4, 5],
    relevance_scores={4: 1.0, 5: 1.0}
)

# 통계 출력
stats = dataset.get_statistics()
print("📊 평가 데이터셋 통계:\n")
for key, value in stats.items():
    print(f"  {key}: {value}")

print("\n💡 평가 데이터셋 구축 팁:")
print("1. 다양한 난이도의 쿼리 포함")
print("2. 실제 사용자 질문 기반")
print("3. 전문가의 관련성 판단")
print("4. 정기적인 업데이트")
print("5. 편향 제거")

📊 평가 데이터셋 통계:

  num_queries: 2
  num_documents: 5
  avg_relevant_per_query: 2.0
  total_relevance_judgments: 4

💡 평가 데이터셋 구축 팁:
1. 다양한 난이도의 쿼리 포함
2. 실제 사용자 질문 기반
3. 전문가의 관련성 판단
4. 정기적인 업데이트
5. 편향 제거


---

## 11. 실전: 문서 ID 기반 평가 함수

지금까지 배운 지표들을 실제 RAG 시스템에서 바로 사용할 수 있는 형태로 구현합니다.

### 이론 vs 실전 함수 비교

| 구분 | 이론용 | 실전용 |
|------|--------|--------|
| 입력 | rankings, relevances | retrieved_docs, relevant_docs |
| 목적 | 개념 이해 | RAG 시스템 통합 |
| 사용 | 학습, 수식 검증 | 실제 평가 파이프라인 |

In [7]:
# 실전용 MRR 함수
def calculate_mrr_practical(retrieved_docs: List[str], relevant_docs: List[str]) -> float:
    """
    MRR 계산 - 문서 ID 기반 실전 버전

    Args:
        retrieved_docs: 검색된 문서 ID 리스트 (순서대로)
        relevant_docs: 관련 문서(정답) ID 리스트

    Returns:
        Reciprocal Rank (0~1, 정답이 없으면 0)
    """
    relevant_set = set(relevant_docs)

    for rank, doc_id in enumerate(retrieved_docs, start=1):
        if doc_id in relevant_set:
            return 1.0 / rank

    return 0.0


# 실전용 nDCG 함수
def calculate_ndcg_practical(retrieved_docs: List[str], relevant_docs: List[str], k: int = None) -> float:
    """
    nDCG 계산 - 문서 ID 기반 Binary Relevance 버전

    Args:
        retrieved_docs: 검색된 문서 ID 리스트 (순서대로)
        relevant_docs: 관련 문서(정답) ID 리스트
        k: 상위 k개만 고려

    Returns:
        nDCG 점수 (0~1)
    """
    if k is None:
        k = len(retrieved_docs)

    relevant_set = set(relevant_docs)

    # Binary relevance 계산
    relevances = [1 if doc in relevant_set else 0 for doc in retrieved_docs[:k]]

    # DCG 계산
    dcg = calculate_dcg(relevances, k)

    # IDCG 계산
    ideal_relevances = sorted(relevances, reverse=True)
    idcg = calculate_dcg(ideal_relevances, k)

    if idcg == 0:
        return 0.0

    return dcg / idcg


# 예시
retrieved = ["doc_A", "doc_B", "doc_C", "doc_D", "doc_E"]
relevant = ["doc_C", "doc_F"]

mrr = calculate_mrr_practical(retrieved, relevant)
ndcg = calculate_ndcg_practical(retrieved, relevant, k=5)

print(f"검색된 문서: {retrieved}")
print(f"정답 문서: {relevant}")
print(f"\nMRR: {mrr:.3f} (첫 정답 doc_C가 3번째)")
print(f"nDCG@5: {ndcg:.3f}")

검색된 문서: ['doc_A', 'doc_B', 'doc_C', 'doc_D', 'doc_E']
정답 문서: ['doc_C', 'doc_F']

MRR: 0.333 (첫 정답 doc_C가 3번째)
nDCG@5: 0.500


### 12. RetrievalEval 종합 클래스

모든 검색 평가 지표를 하나로 묶은 Pydantic 모델입니다. 실제 RAG 파이프라인에서 검색 품질을 종합적으로 평가할 때 사용합니다.

In [8]:
from pydantic import BaseModel, Field

class RetrievalEval(BaseModel):
    """
    검색 평가 결과 모델
    
    RAG 시스템의 검색 단계 성능을 종합적으로 평가합니다.
    """
    mrr: float = Field(..., ge=0, le=1, description="Mean Reciprocal Rank")
    ndcg: float = Field(..., ge=0, le=1, description="Normalized DCG")
    keyword_coverage: float = Field(..., ge=0, le=1, description="키워드 커버리지")
    retrieved_count: int = Field(..., ge=0, description="검색된 문서 수")
    relevant_found: int = Field(..., ge=0, description="찾은 관련 문서 수")

    @property
    def overall_score(self) -> float:
        """종합 점수 (가중 평균)"""
        return 0.4 * self.mrr + 0.4 * self.ndcg + 0.2 * self.keyword_coverage


def evaluate_retrieval(
    retrieved_docs: List[str],
    relevant_docs: List[str],
    retrieved_content: str,
    keywords: List[str],
    k: int = 5
) -> RetrievalEval:
    """
    검색 평가 수행

    Args:
        retrieved_docs: 검색된 문서 ID 리스트
        relevant_docs: 관련 문서(정답) ID 리스트
        retrieved_content: 검색된 문서들의 내용
        keywords: 핵심 키워드 리스트
        k: nDCG 계산에 사용할 k

    Returns:
        RetrievalEval 객체
    """
    relevant_set = set(relevant_docs)
    relevant_found = sum(1 for doc in retrieved_docs if doc in relevant_set)

    return RetrievalEval(
        mrr=calculate_mrr_practical(retrieved_docs, relevant_docs),
        ndcg=calculate_ndcg_practical(retrieved_docs, relevant_docs, k),
        keyword_coverage=calculate_keyword_coverage(retrieved_content, keywords),
        retrieved_count=len(retrieved_docs),
        relevant_found=relevant_found
    )


# 종합 평가 예시
eval_result = evaluate_retrieval(
    retrieved_docs=["doc_A", "doc_B", "doc_C", "doc_D", "doc_E"],
    relevant_docs=["doc_A", "doc_C"],
    retrieved_content="Python에서 sort()와 sorted() 함수를 사용하여 리스트를 정렬할 수 있습니다.",
    keywords=["sort", "sorted", "리스트", "정렬"],
    k=5
)

print("검색 평가 결과:")
print(f"  MRR: {eval_result.mrr:.3f}")
print(f"  nDCG@5: {eval_result.ndcg:.3f}")
print(f"  Keyword Coverage: {eval_result.keyword_coverage:.1%}")
print(f"  관련 문서 발견: {eval_result.relevant_found}/{eval_result.retrieved_count}개")
print(f"\n종합 점수: {eval_result.overall_score:.3f}")

NameError: name 'calculate_keyword_coverage' is not defined

In [ ]:
# 지표 선택 가이드 테이블
guide_data = [
    {
        '상황': '단일 정답을 빠르게 찾기',
        '추천 지표': 'MRR',
        '이유': '첫 번째 정답의 위치가 중요',
        '예시': 'FAQ 검색, 정의 찾기'
    },
    {
        '상황': '여러 관련 문서의 순서 중요',
        '추천 지표': 'nDCG',
        '이유': '관련성 등급과 순서를 함께 고려',
        '예시': '제품 추천, 뉴스 검색'
    },
    {
        '상황': '정확한 결과만 필요',
        '추천 지표': 'Precision@K',
        '이유': '노이즈를 최소화해야 함',
        '예시': '의료 정보, 법률 문서'
    },
    {
        '상황': '모든 관련 문서를 찾아야 함',
        '추천 지표': 'Recall@K',
        '이유': '놓치는 정보가 없어야 함',
        '예시': '특허 검색, 연구 논문'
    },
    {
        '상황': '균형잡힌 성능 필요',
        '추천 지표': 'F1-Score@K',
        '이유': 'Precision과 Recall의 균형',
        '예시': '일반적인 RAG 시스템'
    },
    {
        '상황': '빠른 검색 품질 확인',
        '추천 지표': 'Keyword Coverage',
        '이유': 'LLM 없이 빠르게 확인 가능',
        '예시': '개발/디버깅, 비용 절감'
    },
    {
        '상황': '전체 랭킹 품질 평가',
        '추천 지표': 'MAP',
        '이유': '순서를 고려한 종합 평가',
        '예시': '검색 엔진 평가'
    },
]

df_guide = pd.DataFrame(guide_data)

# 테이블 시각화
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['<b>상황</b>', '<b>추천 지표</b>', '<b>이유</b>', '<b>예시</b>'],
        fill_color='#4CAF50',
        align='left',
        font=dict(color='white', size=12)
    ),
    cells=dict(
        values=[df_guide[col] for col in df_guide.columns],
        fill_color=['#f1f1f1', '#e8f5e9', '#f1f1f1', '#e8f5e9'],
        align='left',
        height=30
    )
)])

fig.update_layout(
    title='RAG 평가 지표 선택 가이드',
    height=450,
    margin=dict(l=0, r=0, t=30, b=0)
)

fig.show()

---

## 13. 실전 팁과 베스트 프랙티스

### RAG 평가 체크리스트

**평가 준비**
- 대표성 있는 쿼리 세트 준비 (최소 50개)
- 전문가의 관련성 판단 확보
- 다양한 난이도와 유형의 쿼리 포함
- 실제 사용 환경과 유사한 조건 설정

**지표 선택**
- 비즈니스 목표에 맞는 주요 지표 선정
- 보조 지표 2-3개 추가 선정
- 사용자 만족도와의 상관관계 검증

**실험 설계**
- A/B 테스트 설정
- 통계적 유의성 검증 계획
- 온라인/오프라인 평가 병행

**결과 분석**
- 실패 케이스 분석 (Error Analysis)
- 개선 포인트 도출
- 사용자 피드백 수집

**최적화**
- 청킹 전략 조정
- 임베딩 모델 비교
- 검색 알고리즘 개선
- 하이브리드 검색 적용

### 추가 개선 아이디어

1. **하이브리드 검색**: BM25 + Vector Search 조합
2. **재순위화**: Cross-Encoder로 상위 결과 재정렬
3. **쿼리 확장**: 동의어/관련어 추가
4. **적응형 K**: 쿼리 복잡도에 따라 K값 동적 조정
5. **캐싱**: 자주 검색되는 쿼리 결과 캐싱
6. **피드백 루프**: 사용자 클릭 데이터로 지속 개선

---

## 14. 요약 및 결론

### 핵심 정리

| 지표 | 핵심 질문 | 범위 | 복잡도 | 우선 사용 |
|------|-----------|------|--------|-----------|
| MRR | 첫 정답이 몇 번째? | 0~1 | 낮음 | 단일 정답 |
| nDCG | 관련성 높은 게 위에? | 0~1 | 높음 | 랭킹 품질 |
| Precision@K | K개 중 정답 비율? | 0~1 | 낮음 | 정확도 중시 |
| Recall@K | 전체 정답 중 찾은 비율? | 0~1 | 낮음 | 완전성 중시 |
| F1@K | P와 R의 균형은? | 0~1 | 낮음 | 균형 |
| Keyword Coverage | 키워드가 얼마나 포함? | 0~1 | 낮음 | 빠른 확인 |
| MAP | 전체 순위 품질은? | 0~1 | 중간 | 종합 평가 |
| Hit Rate | 최소 1개는 찾았나? | 0~1 | 낮음 | 기본 체크 |

### 핵심 메시지

1. **하나의 지표로는 부족합니다** - 여러 지표를 종합적으로 봐야 합니다
2. **비즈니스 목표가 우선입니다** - 사용자 만족도와 연결되는 지표를 선택하세요
3. **지속적인 평가가 중요합니다** - 정기적으로 평가하고 개선하세요
4. **실제 데이터로 평가하세요** - 합성 데이터보다 실제 사용자 쿼리가 중요합니다

> 모든 지표는 **1에 가까울수록 좋은 성능**을 의미합니다!

---

## 참고 자료

- [Information Retrieval 교과서](https://nlp.stanford.edu/IR-book/)
- [TREC (Text REtrieval Conference)](https://trec.nist.gov/)
- [MS MARCO 데이터셋](https://microsoft.github.io/msmarco/)
- [BEIR Benchmark](https://github.com/beir-cellar/beir)
- [LangChain Evaluation](https://python.langchain.com/docs/guides/evaluation)
- [Ragas: RAG 평가 프레임워크](https://github.com/explodinggradients/ragas)

---

**축하합니다! 🎉**

RAG 시스템 평가 지표를 완벽하게 마스터했습니다.
이제 여러분의 RAG 시스템을 정확하게 평가하고 개선할 수 있습니다!